In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import re
import json
import random

In [2]:
# url = 'https://ucsd.edu/catalog/front/courses.html'
# soup = BeautifulSoup(requests.get(url).content, 'html.parser')
# print(soup.prettify())

In [3]:
# cat_string = 'https://ucsd.edu/catalog'
# depts = [str(elem).split('\"')[1].replace('..', cat_string) for elem in soup.find_all('a', text = 'courses')]
# depts

In [4]:
# total = pd.DataFrame()

# for url in depts:
#     soup = BeautifulSoup(requests.get(url).content, 'html.parser')
#     names = soup.find_all(class_='course-name')

#     desc = []
#     for tag in names:
#         desc.append(tag.find_next('p').text)
#     names = [i.text for i in names]
#     df = pd.DataFrame()

#     codes = []
#     departments = []
#     titles = []
#     units = []
#     descriptions = []
#     prereqs = []
#     for i in range(len(names)):
#         name = names[i]
#         print(name)
#         codes.append(name.split('.')[0].split('or')[0])
#         departments.append(name.split('.')[0].split()[0])
#         try:
#             titles.append(' '.join(name.split('.')[1].split()))
#         except:
#             titles.append(' '.join(name.split()[:2]))

#         try:
#             units.append(int(name.split('(')[1][0]))
#         except:
#             units.append(np.nan)

#         description = desc[i]
#         descriptions.append(' '.join(description.split()))

#         try:
#             prereqs.append(' '.join(description.split('Prerequisites:')[1].replace('.', '').split()))
#         except:
#             prereqs.append(None)


#     df['code'] = codes
#     df['dept'] = departments
#     df['title'] = titles
#     df['min_units'] = units
#     df['description'] = descriptions
#     df['prereqs'] = prereqs
#     total = total.append(df)
#     print(url)
#     time.sleep(1)

In [5]:
# total

In [6]:
# total.to_csv('all_courses.csv')

In [7]:
df = pd.read_csv('data/all_courses.csv').drop(columns = ['Unnamed: 0'])
df

,code,dept,title,min_units,description,prereqs
0,AIP 97,AIP,"Academic Internship (2, 4)",2.0,Individual placements for field learning. Must...,"lower-division standing, completion of thirty ..."
1,AIP 197,AIP,"Academic Internship Program (2, 4, 6, 8, 10, 12)",2.0,Individual internship placements integrated wi...,upper-division standing; department approval
2,AIP 197DC,AIP,"UCDC: Washington, DC Internship (6, 8, 10)",6.0,This internship is attached to the University ...,upper-division standing; department approval
3,AIP 197F,AIP,"Academic Internship Program—Foreign (4, 8, 12)",4.0,Individual placements for field learning oppor...,ninety units completed; 25 minimum cumulative ...
4,AIP 197P,AIP,"Public Service Internship (4, 8, 12)",4.0,Individual placements for field learning perfo...,ninety units completed; 25 minimum cumulative ...
...,...,...,...,...,...,...
6302,WCWP 10B,WCWP,The Writing Course B (4),4.0,A seminar-style course in reading and writing ...,satisfaction of WCWP 10A and must be a Warren ...
6303,WCWP 100,WCWP,Academic Writing (4),4.0,An upper-division workshop course in argumenta...,junior/senior standing and must be a Warren Co...
6304,WCWP 160,WCWP,Technical Writing for Scientists and Engineers...,4.0,An upper-division workshop-style writing cours...,junior/senior standing
6305,WARR 189,WARR,Academic Mentoring and the Writing Process (2),2.0,Students will gain a fundamental understanding...,permission of instructor is required to enroll


In [8]:
# df['dept'].apply(len).value_counts()

In [9]:
unique = df['dept'].unique()

def prereqs_to_list(prereqs):
    if isinstance(prereqs, float):
        return []
    out_list = []
    
    # explicitly named courses
    for dept in unique:
        if dept in prereqs:
            for code in df[df['dept'] == dept]['code']:
                if re.search(code + '\W', prereqs):
                    out_list.append(code)
                    
    # or/and (as in MATH 11 or 18)
    or_and = re.compile('([A-Z]{2,} [0-9]+[A-Z]* (or|and) [0-9]+[A-Z]*)')
    more_courses = [i[0] for i in re.findall(or_and, prereqs)]
    new_matches = []
    for match in more_courses:
        dept = match.split()[0]
        code = match.split()[-1]
        new_matches.append(dept + ' ' + code)
    
    # hyphens
    hyphens = re.compile('([A-Z]{2,} [0-9]+[A-Z]*\s*(–|-)\s*[0-9]*[A-Z]*)')
    more_courses = [i[0] for i in re.findall(hyphens, prereqs)]
    for match in more_courses:
        dept = match.split()[0]
        code = match.split()[-1]
        if '-' in code:
            code_letter = code.split('-')[-1]
        else:
            code_letter = code.split('–')[-1]
        code_start = (re.findall('^[0-9]+', code)[0])
        new_match = dept + ' ' + code_start + code_letter
        new_matches.append(new_match)
    
    for match in new_matches:
        if match in df['code']:
            out_list.append(match)
    
    return list(set(out_list))

In [10]:
lists = df['prereqs'].apply(prereqs_to_list)

In [11]:
lists

0               []
1               []
2               []
3               []
4               []
           ...    
6302    [WCWP 10A]
6303            []
6304            []
6305            []
6306            []
Name: prereqs, Length: 6307, dtype: object

In [12]:
df['prereq_list'] = lists
df

,code,dept,title,min_units,description,prereqs,prereq_list
0,AIP 97,AIP,"Academic Internship (2, 4)",2.0,Individual placements for field learning. Must...,"lower-division standing, completion of thirty ...",[]
1,AIP 197,AIP,"Academic Internship Program (2, 4, 6, 8, 10, 12)",2.0,Individual internship placements integrated wi...,upper-division standing; department approval,[]
2,AIP 197DC,AIP,"UCDC: Washington, DC Internship (6, 8, 10)",6.0,This internship is attached to the University ...,upper-division standing; department approval,[]
3,AIP 197F,AIP,"Academic Internship Program—Foreign (4, 8, 12)",4.0,Individual placements for field learning oppor...,ninety units completed; 25 minimum cumulative ...,[]
4,AIP 197P,AIP,"Public Service Internship (4, 8, 12)",4.0,Individual placements for field learning perfo...,ninety units completed; 25 minimum cumulative ...,[]
...,...,...,...,...,...,...,...
6302,WCWP 10B,WCWP,The Writing Course B (4),4.0,A seminar-style course in reading and writing ...,satisfaction of WCWP 10A and must be a Warren ...,[WCWP 10A]
6303,WCWP 100,WCWP,Academic Writing (4),4.0,An upper-division workshop course in argumenta...,junior/senior standing and must be a Warren Co...,[]
6304,WCWP 160,WCWP,Technical Writing for Scientists and Engineers...,4.0,An upper-division workshop-style writing cours...,junior/senior standing,[]
6305,WARR 189,WARR,Academic Mentoring and the Writing Process (2),2.0,Students will gain a fundamental understanding...,permission of instructor is required to enroll,[]


In [13]:
colormap = {}
for dept in df['dept'].unique():
    colormap[dept] = '#%02x%02x%02x' % tuple(random.choice(range(32,256,32)) for _ in range(3))
df['color'] = df['dept'].apply(lambda x: colormap[x])

In [14]:
# df.to_csv('courses_with_prereqs.csv')

In [15]:
# df = pd.read_csv('courses_with_prereqs.csv').drop(columns = ['Unnamed: 0'])
# df

In [16]:
df['fulltitle'] = df['code'].str.cat(df['title'], sep = '. ')

In [17]:
nodes = df[['code', 'fulltitle']].copy(deep = True).drop_duplicates(subset = ['code'])
nodes.columns = ['id', 'label']
nodes.to_csv('data/nodes.csv', index = False)
len(nodes)

6271

In [18]:
titles = df[['code', 'title']].drop_duplicates(subset = ['code'])['title']
colors = df[['code', 'color']].drop_duplicates(subset = ['code'])['color']

In [19]:
edges = []
for i in range(len(df)):
    row = df.iloc[i]
    for j in row['prereq_list']:
        edges.append({'source' : j, 'target' : row['code']})

In [20]:
pd.DataFrame(edges).drop_duplicates().to_csv('data/edges.csv', index = False)

In [21]:
edge_df = pd.DataFrame(edges)
out_degree = edge_df['source'].value_counts()
out_degree

MATH 18      65
COMM 10      60
MATH 31AH    56
COMM 100A    47
MATH 20C     36
             ..
CLRE 254      1
EDS 117       1
CHIN 20BD     1
CSE 5A        1
PHIL 110      1
Name: source, Length: 1062, dtype: int64

In [113]:
def get_out_degree(course):
    try:
        return 1 + 4 * out_degree[course]
    except:
        return 1

nodes['size'] = nodes['id'].apply(get_out_degree)
nodes['x'] = [random.randint(0,500) for i in range(len(nodes))]
nodes['y'] = [random.randint(0,500) for i in range(len(nodes))]
nodes['title'] = titles
nodes['dept'] = nodes['id'].str.split().str.get(0)
nodes['color'] = colors

In [114]:
node_json = json.loads(nodes.to_json(orient = 'records'))

In [115]:
edge_df['id'] = edge_df.index
edge_df['source_dept'] = edge_df['source'].str.split().str.get(0)
edge_df['target_dept'] = edge_df['target'].str.split().str.get(0)
edge_df['type'] = ['arrow'] * len(edge_df)
edge_json = json.loads(edge_df.to_json(orient = 'records'))

In [116]:
graph_json = {
    'nodes' : node_json,
    'edges' : edge_json
}

json_3d = {
    'nodes' : node_json,
    'links' : edge_json
}

In [117]:
with open("data/data.js", "w") as outfile:  
    outfile.write('let data = ' + json.dumps(graph_json))
    
with open("data/data_3d.js", "w") as outfile:  
    outfile.write('let data_3d = ' + json.dumps(json_3d))

In [118]:
out_degree[0:15]

MATH 18      65
COMM 10      60
MATH 31AH    56
COMM 100A    47
MATH 20C     36
PHYS 2A      35
MATH 20D     35
LTSP 50A     35
MATH 20A     30
BILD 1       29
CSE 11       22
MATH 180A    22
CSE 100      21
MATH 10A     21
CSE 8B       20
Name: source, dtype: int64